In [1]:
import tensorflow as tf

import cv2

import numpy as np

import os

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense,Flatten,Conv2D,Activation,Dropout

from keras import backend as K

import keras

from keras.models import Sequential, Model

from keras.models import load_model

from keras.optimizers import SGD

from keras.callbacks import EarlyStopping,ModelCheckpoint

from keras.layers import MaxPool2D

from google.colab.patches import cv2_imshow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import shutil
shutil.unpack_archive("/content/drive/MyDrive/breakfastType.zip", "breakfastType/")

In [4]:
train_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory("/content/breakfastType/breakfastType/train",target_size=(224, 224),batch_size=32,shuffle=True,class_mode='categorical')

test_generator = test_datagen.flow_from_directory("/content/breakfastType/breakfastType/test",target_size=(224,224),batch_size=32,shuffle=False,class_mode='categorical')

Found 2088 images belonging to 4 classes.
Found 54 images belonging to 4 classes.


In [5]:
def VGG16():

  model = Sequential()

  model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))

  model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))

  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))

  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))

  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))

  model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))

  model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))

  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

  model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))

  model.add(Flatten(name='flatten'))

  model.add(Dense(256, activation='relu', name='fc1'))

  model.add(Dense(128, activation='relu', name='fc2'))

  model.add(Dense(4, activation='softmax', name='output'))

  return model

In [6]:

model=VGG16()

model.summary()

Vgg16 = Model(inputs=model.input, outputs=model.get_layer('vgg16').output)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                   

In [7]:
es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)

In [8]:
opt = SGD(learning_rate=1e-6, momentum=0.9)

model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

In [9]:
for layer in Vgg16.layers:

    layer.trainable = False

for layer in model.layers:

    print(layer, layer.trainable)

<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7ff30db1d900> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7ff30dfdb9d0> False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7ff30dfdb6d0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7ff305f18dc0> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7ff305f19540> False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7ff30db1e590> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7ff305f1a350> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7ff305f1b370> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7ff305f1b3a0> False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7ff305f1b190> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7ff305f85630> False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7ff305f86830> False
<keras.src.layers.convolutional.conv2d.Conv

In [10]:
mc = ModelCheckpoint('/content/drive/MyDrive/vgg16_best_modeli_1.h5', monitor='val_accuracy', mode='max', save_best_only=True)

In [11]:
H = model.fit_generator(train_generator,validation_data=test_generator,epochs=50,verbose=1,callbacks=[mc])

<ipython-input-11-a80656c21635>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(train_generator,validation_data=test_generator,epochs=50,verbose=1,callbacks=[mc])


Epoch 1/50
66/66 [==============================] - ETA: 0s - loss: 1.2912 - accuracy: 0.3338

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


66/66 [==============================] - 77s 821ms/step - loss: 1.2912 - accuracy: 0.3338 - val_loss: 1.4174 - val_accuracy: 0.1481
Epoch 2/50
66/66 [==============================] - 38s 569ms/step - loss: 1.2194 - accuracy: 0.3218 - val_loss: 1.3591 - val_accuracy: 0.1481
Epoch 3/50
66/66 [==============================] - 38s 568ms/step - loss: 1.1810 - accuracy: 0.3611 - val_loss: 1.3205 - val_accuracy: 0.1481
Epoch 4/50
66/66 [==============================] - 41s 624ms/step - loss: 1.1439 - accuracy: 0.4076 - val_loss: 1.2828 - val_accuracy: 0.2037
Epoch 5/50
66/66 [==============================] - 39s 591ms/step - loss: 1.1069 - accuracy: 0.4698 - val_loss: 1.2464 - val_accuracy: 0.2407
Epoch 6/50
66/66 [==============================] - 41s 612ms/step - loss: 1.0753 - accuracy: 0.5354 - val_loss: 1.2009 - val_accuracy: 0.4259
Epoch 7/50
66/66 [==============================] - 41s 612ms/step - loss: 1.0414 - accuracy: 0.5613 - val_loss: 1.1598 - val_accuracy: 0.5185
Epoch 8/50

In [12]:
import tensorflow as tf

model = tf.keras.models.load_model('/content/drive/MyDrive/vgg16_best_modeli_1.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("breakfast.tflite", "wb").write(tflite_model)

84693420

In [13]:
pip install keras.preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 915.5 kB/s eta 0:00:00


In [21]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
img_path = '/content/test.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds=model.predict(x)
# create a list containing the class labels
class_labels = ['tomato','cheese','cucumber','olives']

# find the index of the class with maximum score
pred = np.argmax(preds, axis=-1)

# print the label of the class with maximum score
print(class_labels[pred[0]])

1/1 [==============================] - 0s 18ms/step
olives
